In [7]:
import xml.dom.minidom as minidom
import xml.etree.ElementTree as xmlET
from future.utils import iteritems

from pyPRMS.prms_helpers import read_xml

In [20]:
parent_modules = {'potet': ['potet_hamon', 'potet_hs', 'potet_jh', 'potet_pan', 'potet_pm', 'potet_pm_sta', 'potet_pt', 'potet_hru'],
                  'solar_radiation': ['ccsolrad', 'ddsolrad', 'solar_radiation_hru'], #??soltab??
                  'streamflow': ['muskingum', 'muskingum_lake', 'strmflow_in_out'], 
                  'temperature': ['temp_1sta', 'temp_laps', 'temp_dist2', 'temperature_hru'],
                  'precipitation': ['precip_1sta', 'precip_laps', 'precip_dist2', 'precipitation_hru'],
                  'transpiration': ['transp_frost', 'transp_tindex']}

In [23]:
# Read in the parameters.xml file
global_params_file = '/Users/pnorton/tmp/parameters.xml'
params_root = read_xml(global_params_file)

module_dict = {}
dims_by_param = {}

# Populate parameterSet with all available parameter names
for param in params_root.findall('parameter'):
    xml_param_name = param.attrib.get('name')

    # Build dictionary of dimensions (in correct order) for each parameter
    for cdim in param.findall('./dimensions/dimension'):
        dim_name = cdim.attrib.get('name')
        dim_pos = int(cdim.find('position').text)
        dims_by_param.setdefault(xml_param_name, []).insert(dim_pos, dim_name)    
    
    mods_list = []
    for cmod in param.findall('./modules/module'):
        mods_list.append(cmod.text)
        
    in_parent = False
    
    # If a parameter is required by all the child modules of a parent then add
    # only the parent module to the dictionary.
    for pp, tt in iteritems(parent_modules):
        if len(set(tt).difference(set(mods_list))) == 0:
            module_dict.setdefault(pp, []).append(xml_param_name)
            in_parent = True
            
            # There could be other child modules remaining so add them too
            for mm in set(tt).symmetric_difference(set(mods_list)):
                module_dict.setdefault(mm, []).append(xml_param_name)
                
            break
    
    if not in_parent:
        for mm in mods_list:
            module_dict.setdefault(mm, []).append(xml_param_name)

In [24]:
# List the parameters for each module
for xx, yy in iteritems(module_dict):
    print(xx)
    for zz in yy:
        print('\t{}'.format(zz))

precipitation_hru
	rain_cbh_adj
	snow_cbh_adj
muskingum_lake
	segment_flow_init
	elev_outflow
	lake_coef
	lake_din1
	lake_init
	lake_out2
	lake_out2_a
	lake_out2_b
	lake_qro
	lake_segment_id
	lake_vol_init
	nsos
	o2
	obsout_lake
	rate_table
	rate_table2
	rate_table3
	rate_table4
	ratetbl_lake
	s2
	tbl_gate
	tbl_gate2
	tbl_gate3
	tbl_gate4
	tbl_stage
	tbl_stage2
	tbl_stage3
	tbl_stage4
	weir_coef
	weir_len
convert_params
	hru_area
	tmax_allrain_offset
	tmax_allsnow
	sat_threshold
	soil_moist_init_frac
	soil_moist_max
	soil_rechr_init_frac
	soil_rechr_max_frac
	ssstor_init_frac
gwflow
	elevlake_init
	gw_seep_coef
	gwflow_coef
	gwsink_coef
	gwstor_init
	gwstor_min
	lake_seep_elev
transp_tindex
	transp_beg
	transp_end
	transp_tmax
ide_dist
	psta_elev
	tmax_adj
	tmin_adj
potet_hs
	hs_krs
temperature_hru
	tmax_cbh_adj
	tmin_cbh_adj
temp_1sta
	hru_tsta
	basin_tsta
	tsta_elev
	tmax_adj
	tmin_adj
frost_date
	frost_temp
srunoff
	carea_max
	carea_min
	dprst_depth_avg
	dprst_et_coef
	dprst_flow_co

In [12]:
# List dimensions for each parameter
for xx, yy in iteritems(dims_by_param):
    print(xx, yy)

('psta_mon', ['nrain', 'nmonths'])
('padj_sn', ['nrain', 'nmonths'])
('lapsemax_max', ['nmonths'])
('tbl_gate3', ['ngate3'])
('tbl_gate2', ['ngate2'])
('ccov_intcp', ['nhru', 'nmonths'])
('tbl_gate4', ['ngate4'])
('width_alpha', ['nsegment'])
('tstorm_mo', ['nhru', 'nmonths'])
('gw_up_id', ['ncascdgw'])
('parent_ssr', ['nhru'])
('hru_strmseg_down_id', ['ncascade'])
('lat_temp_adj', ['nsegment', 'nmonths'])
('va_open_exp', ['nhru'])
('tbl_gate', ['ngate'])
('nhm_seg', ['nsegment'])
('gw_strmseg_down_id', ['ncascdgw'])
('hru_up_id', ['ncascade'])
('jh_coef_hru', ['nhru'])
('snow_mon', ['nhru', 'nmonths'])
('nhm_id', ['nhru'])
('lapsemin_max', ['nmonths'])
('pref_flow_den', ['nhru'])
('weir_len', ['nlake'])
('snarea_curve', ['ndeplval'])
('vdemx', ['nsegment'])
('rad_conv', ['one'])
('covden_sum', ['nhru'])
('hru_percent_imperv', ['nhru'])
('soil2gw_max', ['nhru'])
('dprst_frac', ['nhru'])
('pm_d_coef', ['nhru', 'nmonths'])
('hs_krs', ['nhru', 'nmonths'])
('rain_code', ['nmonths'])
('temp

### Write parameters into csv format

In [5]:
import pyPRMS.NhmParamDb_v2 as nhm
import pandas as pd

In [6]:
workdir = '/Users/pnorton/Projects/National_Hydrology_Model/datasets/bandit/paramdb_v2'

pdb = nhm.NhmParamDb_v2(workdir)

In [17]:
df = pdb.parameters.get('dday_slope').as_dataframe

In [12]:
pdb.parameters.get('dday_slope').dimensions.tostructure()

{'nhru': {'position': 0, 'size': 109951},
 'nmonths': {'position': 1, 'size': 12}}

In [21]:
# Create a position-ordered list of dimension names for the parameter
dim_list = []

for dm, dd in iteritems(pdb.parameters.get('dday_slope').dimensions.tostructure()):
    dim_list.insert(dd['position'], dm)

print(','.join(dim_list))

nhru,nmonths


In [20]:
# Write a CSV file of parameter data with the dimension names as a first-line comment

fhdl = open('dday_slope.csv', 'w')

# Write the dimension info as first line
fhdl.write('# {}\n'.format(','.join(dim_list)))

# Write the data to a csv file
df.index += 1
df.to_csv(fhdl, header=False, index=True)

fhdl.close()